In [1]:
!pip uninstall -y transformers accelerate

Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0


In [2]:
!pip install transformers==4.36.2 accelerate==0.25.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl (8.2 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f7/fc/c55e5a2da345c9a24aa2e1e0f60eb2ca290b6a41be82da03a6d4baec4f99/accelerate-0.25.0-py3-none-any.whl (265 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate] 1/2 [accelerate]


In [3]:
import os
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [4]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [6]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [7]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [8]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(dataset["train"])

,label,text
0,4 stars,"My roommate took his 09 Honda Civic in for a brake job. Job was done today as promised, within estimated amount, if at the high end for a single axle. Whole facility, including garage area, was super clean. Comfortable, attractive waiting area with wide screen TV. Separate small play area for small children within sight with it's own TV and DVD player. As others have mentioned, there's a cooler with free bottled water as well as a single cup coffee maker and complimentary coffee. Man at the counter was courteous. \""Our\"" first experience here was routine and went fine, with no complaints, and nothing exceptional either way, which would ordinarily rate a perfectly complimentary 3 stars from me, but I'm going to go with a 4 based on the amenities and the impressive cleanliness of the entire facility, including the garage bays which you can see from the picture window behind the front desk. \n\nAlthough I have a garage I like and trust in mid-town Phoenix, the experience here, proximity to home, and the convenient Saturday hours would bring me back to Honest-1 with my own car should the need arise. With a little more experience which (nothing against you, o Honest-1), I hope I don't have need of, I might be willing to say \""it's as good as it gets\"". For now, it was an A-OK experience, with bonus points for cleanliness, and I would go back."
1,5 stars,"Beautiful selections!\nGreat prices! Wonderful service, wonderful alterations!\nI got my wedding dress here, it was ready when they promised! It looked beautiful and I think I got a great deal!"
2,5 stars,This is certainly my favorite place in the valley to get good Thai food! The flavors are fantastic! I never leave still hungry!
3,2 star,"I hate to only give this Olive Garden 2 stars because my husband and I have eaten at this location twice now and had very sweet waitresses both times but the food continues to disappoint us. I want to like this Olive Garden because we've received quite a few gift cards to the chain, but the food was bland and below average and the \""endless\"" salad looked and tasted like it was old and slightly browning. With so many other great options at this price point in the Charlotte area, we will not bother coming back for round three."
4,1 star,"Cute atmosphere, horrible service. We sat down at a table and waited 25 minutes. We were never acknowledged even though the waitress sat down at the table next to us and had a conversation with the party. She came and went at least 3 times and never stopped to ask if we had been helped."
5,4 stars,"Hubby and I had such a great time at Vegas, and this hotel was great! Service was with a smile from everyone (bell hop, to registration, to room service) - everyone was helpful! We were given a strip view for no extra charge, and the room was immaculate! \n\nI gave 4 stars because of its proximity on the strip, but it is still an awesome hotel nonetheless. Next time, I'd like to stay at the Wynn so I can compare between the sister hotels!"
6,4 stars,"Came here on a weekday for an escape and catch up with my uncle and was very pleasantly surprised. I'd been to a Bahama Breeze near Seattle a couple years ago and it screamed chain. So did this one upon first inspection, but a few things stood out.\n\nFirst, our waiter was top notch. Friendly, engaging, knowledgeable, and even engaged us as we asked about how the local dining businesses were faring in the current economic state. He was actually a PT waiter at another restaurant, and he helped us pick our dishes.\n\nThe food was surprisingly tasty. I had the jerk chicken bowtie pasta, which was slightly spicy but in a good way.\n\nAlthough I might not go actively searching for this place next time I am in Vegas, I would recommend it if you're looking for a reasonably priced, tasty meal."
7,4 stars,The music and acting was wonderful. We are glad that we had the opportunity to catch it before it left. I do agree that it seemed to drag a little.
8,

# 预处理数据

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,1 star,"OMG! So I call this restaurant for delivery and they pick up and hang up...Ordinarily I would say it was an accident but after the 5th call and they hung up, of course I just kept doing it until i got a fast busy....awful","[101, 152, 14666, 106, 1573, 146, 1840, 1142, 4382, 1111, 6779, 1105, 1152, 3368, 1146, 1105, 7311, 1146, 119, 119, 119, 2926, 18140, 11486, 146, 1156, 1474, 1122, 1108, 1126, 4216, 1133, 1170, 1103, 4025, 1840, 1105, 1152, 4629, 1146, 117, 1104, 1736, 146, 1198, 2023, 1833, 1122, 1235, 178, 1400, 170, 2698, 5116, 119, 119, 119, 119, 9684, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


# 数据抽样

In [12]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
all_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
all_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

# 微调训练配置
# 加载 BERT 模型

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # 检查是否支持 GPU


2.7.0+cu128
True


# 训练超参数（TrainingArguments）

In [15]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [16]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=

# 训练过程中的指标评估（Evaluate)

In [17]:
!pip install evaluate

In [18]:
!pip install scikit-learn

In [19]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=model_dir,
    evaluation_strategy="epoch", 
    per_device_train_batch_size=16,
    num_train_epochs=3,
    logging_steps=30,
    save_strategy="no"
)

# 开始训练

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=all_train_dataset,
    eval_dataset=all_eval_dataset,
    compute_metrics=compute_metrics,
)

In [23]:
!nvidia-smi

Mon Jul 28 12:40:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.144                Driver Version: 570.144        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 6000                Off |   00000000:B3:00.0  On |                  Off |
| 39%   55C    P0             74W /  260W |    1430MiB /  24576MiB |     21%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.800300,0.755748,0.671120
2,0.696100,0.727051,0.683800
3,0.585700,0.733750,0.693540


TrainOutput(global_step=121875, training_loss=0.7101923191794371, metrics={'train_runtime': 68041.9058, 'train_samples_per_second': 28.659, 'train_steps_per_second': 1.791, 'total_flos': 5.130803778048e+17, 'train_loss': 0.7101923191794371, 'epoch': 3.0})

In [25]:
trainer.save_model(model_dir)